In [1]:
import pymysql.cursors
import pandas as pd
from sklearn import preprocessing

# 训练

## 获取全量的竞彩比赛列表

In [2]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_football_game_list`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_game_list_df = pd.DataFrame(rows)
    
    connection.close()

## 获取全量的胜负彩比赛列表

In [3]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_lottery_info`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_lottery_game_list_df = pd.DataFrame(rows)
    
    connection.close()

## 合并竞彩比赛列表和胜负彩比赛列表

In [4]:
tmp = train_lottery_game_list_df.drop(['issue'], axis=1)
df = pd.concat([train_game_list_df, tmp])
df = df[['matchid', 'game', 'home_team', 'visit_team', 'gs', 'gd', 'gn', 'time', 'result', 'win_bet_return', 'draw_bet_return', 'lose_bet_return']]
df = df.drop_duplicates(subset=['matchid'])

## **设定训练范围** 并处理数据

In [5]:
# match_group = ['澳超', '英超', '德甲', '德乙', '法甲', '西甲', '意甲', '日职', '英甲', '英冠', '苏超', '法乙', '葡超', '荷甲', '荷乙', '韩K联', '瑞典超', '挪超', '美职', '日乙', '俄超', '比甲', '瑞典甲', '法丙', '挪甲', '英乙', '苏冠', '巴甲', '智利甲', '墨超', '智利乙', '阿甲', '欧冠', '欧罗巴']
match_group = ['澳超', '英超', '德甲', '德乙', '法甲', '西甲', '意甲', '日职', '英甲', '英冠', '苏超', '法乙', '葡超', '荷甲', '荷乙', '韩K联', '瑞典超', '挪超', '美职', '日乙', '俄超', '比甲', '瑞典甲', '法丙', '挪甲', '英乙', '苏冠', '巴甲', '智利甲', '墨超', '智利乙', '阿甲']
match_df = df[(df['game'].isin(match_group))]
match_df = match_df.dropna()

In [6]:
# 对team做encode 这个encoder后面预测的时候还会用到
teams = list(set(df['home_team'].values) | set(df['visit_team'].values))
team_encoder = preprocessing.LabelEncoder()
team_encoder.fit(teams)

LabelEncoder()

In [7]:
def encode_team(df):
    df['home_team_encoder'] = team_encoder.transform(df['home_team'])
    df['visit_team_encoder'] = team_encoder.transform(df['visit_team'])
    return df

In [8]:
# 比赛名称encode
games = list(set(match_df['game'].values))
game_encoder = preprocessing.LabelEncoder()
game_encoder.fit(games)

LabelEncoder()

In [9]:
def encode_game(df):
    df['game_encoder'] = game_encoder.transform(df['game'])
    return df

In [10]:
match_df['year'] = match_df.apply(lambda row: row.time.year, axis=1)
match_df['month'] = match_df.apply(lambda row: row.time.month, axis=1)
match_df['day'] = match_df.apply(lambda row: row.time.day, axis=1)
match_df['fix_result'] = match_df.apply(lambda row: int(row.result) if row.result < 3 else 2, axis=1)

In [11]:
match_df = encode_team(match_df)
match_df = encode_game(match_df)

In [12]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44524 entries, 1 to 12549
Data columns (total 19 columns):
matchid               44524 non-null int64
game                  44524 non-null object
home_team             44524 non-null object
visit_team            44524 non-null object
gs                    44524 non-null int64
gd                    44524 non-null int64
gn                    44524 non-null int64
time                  44524 non-null datetime64[ns]
result                44524 non-null int64
win_bet_return        44524 non-null float64
draw_bet_return       44524 non-null float64
lose_bet_return       44524 non-null float64
year                  44524 non-null int64
month                 44524 non-null int64
day                   44524 non-null int64
fix_result            44524 non-null int64
home_team_encoder     44524 non-null int64
visit_team_encoder    44524 non-null int64
game_encoder          44524 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(12), object

## 获取特征数据

In [13]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_football_feature_info`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_feature_df = pd.DataFrame(rows)
    
    connection.close()

## 处理特征数据

In [14]:
def get_count(col):
    arr = col.split(' ')
    if len(arr) == 2:
        return int(arr[1])
    
    return int(arr[0])

In [15]:
def get_count_rate(col, all_count):
    arr = col.split(' ')
    if len(arr) == 2:
        return int(arr[1]) / int(all_count)
    
    return int(arr[0])  / int(all_count)

In [16]:
def get_current_score(words, score, pos):
    t = score.replace(words, '')
    t = t.replace('胜', '')
    t = t.replace('平', '')
    t = t.replace('负', '')
    t = t.replace('分', '')
    
    arr = t.split('  ')
    
    if len(arr) == 3 and len(arr[2].split(' ')) == 2:
        arr = [arr[0], arr[1]] + arr[2].split(' ')
    
    if len(arr) != 4:
        return None
    
    if arr[pos] != '':
        return int(arr[pos])
    else:
        return None

In [17]:
h_cols = ['h_perf_win', 'h_perf_draw', 'h_perf_lose', 'h_host_win', 'h_host_draw', 'h_host_lose', 'h_battle_with_front_10_win', 'h_battle_with_front_10_draw', 'h_battle_with_front_10_lose', 'h_battle_with_end_10_win', 'h_battle_with_end_10_draw', 'h_battle_with_end_10_lose']
v_cols =['v_perf_win', 'v_perf_draw', 'v_perf_lose', 'v_host_win', 'v_host_draw', 'v_host_lose', 'v_battle_with_front_10_win', 'v_battle_with_front_10_draw', 'v_battle_with_front_10_lose', 'v_battle_with_end_10_win', 'v_battle_with_end_10_draw', 'v_battle_with_end_10_lose']

In [18]:
def take_count(df):
    cols = h_cols + v_cols
    for v in cols:
        df[v + '_count'] = df.apply(lambda row: get_count(row[v]), axis=1)
    
    df['h_count'] = df['h_perf_win_count'] + df['h_perf_draw_count'] + df['h_perf_lose_count']
    df['v_count'] = df['v_perf_win_count'] + df['v_perf_draw_count'] + df['v_perf_lose_count']
    
    return df

In [19]:
def take_rate(df):
    for v in h_cols:
        df[v + '_count_rate'] = df.apply(lambda row: get_count_rate(row[v], row.h_count), axis=1)

    for v in v_cols:
        df[v + '_count_rate'] = df.apply(lambda row: get_count_rate(row[v], row.v_count), axis=1)
        
    return df

In [20]:
def take_score(df):
    h_items = ['h_current_win_count', 'h_current_draw_count', 'h_current_lose_count']
    for k, v in enumerate(h_items):
        df[v] = df.apply(lambda row: get_current_score('主队：', row.h_score, k), axis=1)
    
    df['h_all_count'] = df['h_current_win_count'] + df['h_current_draw_count'] + df['h_current_lose_count']
    
    for v in h_items:
        df[v + '_rate'] = df[v] / df['h_all_count']
        
    df['h_current_score'] = df.apply(lambda row: get_current_score('主队：', row.h_score, 3), axis=1)
    
    ###########################################################################################
    v_items = ['v_current_win_count', 'v_current_draw_count', 'v_current_lose_count']
    for k, v in enumerate(v_items):
        df[v] = df.apply(lambda row: get_current_score('客队：', row.v_score, k), axis=1)
    
    df['v_all_count'] = df['v_current_win_count'] + df['v_current_draw_count'] + df['v_current_lose_count']
    
    for v in v_items:
        df[v + '_rate'] = df[v] / df['v_all_count']
        
    df['v_current_score'] = df.apply(lambda row: get_current_score('客队：', row.v_score, 3), axis=1)
    
    return df

In [21]:
def get_current_rank(rank):
    arr = rank.split('  ')
    
    if len(arr) == 2:
        rank_str = arr[1].replace('本赛季排名：', '')
        if rank_str != '':
            return int(rank_str)
    
    return None

In [22]:
def take_goal_info(prefix, df):
    target_cols = ['_perf_gs', '_perf_gd', '_perf_avg_gs', '_perf_avg_gd',
    '_host_gs', '_host_gd', '_host_avg_gs', '_host_avg_gd',
    '_r3_gs', '_r3_gd', '_r3_avg_gs', '_r3_avg_gd',
    '_perf_bet_high', '_perf_bet_low', '_host_bet_high',
    '_host_bet_low', '_host_0_1_goal', '_host_2_3_goal',
    '_host_ab_4_goal', '_host_0_goal', '_host_1_goal',
    '_host_2_goal', '_host_3_goal', '_host_4_goal', '_host_5_goal',
    '_host_6_goal', '_host_7_goal']

    for k in target_cols:
        df[prefix + k + '_rate'] = df[prefix + k] / df[prefix + '_count']
    
    return df

后面对于预测数据可以复用

In [23]:
train_feature_df = take_count(train_feature_df)
train_feature_df = take_rate(train_feature_df)
train_feature_df = take_score(train_feature_df)

train_feature_df['h_current_rank'] = train_feature_df.apply(lambda row: get_current_rank(row.h_rank), axis=1)
train_feature_df['v_current_rank'] = train_feature_df.apply(lambda row: get_current_rank(row.v_rank), axis=1)

train_feature_df = take_goal_info('h', train_feature_df)
train_feature_df = take_goal_info('v', train_feature_df)

## 合并训练数据

In [24]:
train_dataset_df = pd.merge(match_df, train_feature_df, on='matchid', how='left')
train_dataset_df = train_dataset_df.dropna()
train_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40440 entries, 0 to 44533
Columns: 224 entries, matchid to v_host_7_goal_rate
dtypes: datetime64[ns](1), float64(180), int64(12), object(31)
memory usage: 69.4+ MB


## xgb训练

In [25]:
import xgboost as xgb

x_columns = [
    'year', 
    'month', 
    'win_bet_return', 'draw_bet_return','lose_bet_return', 'home_team_encoder', 'visit_team_encoder', 'game_encoder', 
    
    'h_perf_gs', 'h_perf_gd', 'h_perf_avg_gs', 'h_perf_avg_gd',
    'h_host_gs', 'h_host_gd', 'h_host_avg_gs', 'h_host_avg_gd',
    'h_r3_gs', 'h_r3_gd', 'h_r3_avg_gs', 'h_r3_avg_gd',
    'h_perf_bet_high', 'h_perf_bet_low', 'h_host_bet_high',
    'h_host_bet_low', 'h_host_0_1_goal', 'h_host_2_3_goal',
    'h_host_ab_4_goal', 'h_host_0_goal', 'h_host_1_goal',
    'h_host_2_goal', 'h_host_3_goal', 'h_host_4_goal', 'h_host_5_goal',
    'h_host_6_goal', 'h_host_7_goal', 
    
    'h_perf_gs_rate', 'h_perf_gd_rate', 'h_perf_avg_gs_rate', 'h_perf_avg_gd_rate',
    'h_host_gs_rate', 'h_host_gd_rate', 'h_host_avg_gs_rate', 'h_host_avg_gd_rate',
    'h_r3_gs_rate', 'h_r3_gd_rate', 'h_r3_avg_gs_rate', 'h_r3_avg_gd_rate',
    'h_perf_bet_high_rate', 'h_perf_bet_low_rate', 'h_host_bet_high_rate',
    'h_host_bet_low_rate', 'h_host_0_1_goal_rate', 'h_host_2_3_goal_rate',
    'h_host_ab_4_goal_rate', 'h_host_0_goal_rate', 'h_host_1_goal_rate',
    'h_host_2_goal_rate', 'h_host_3_goal_rate', 'h_host_4_goal_rate', 'h_host_5_goal_rate',
    'h_host_6_goal_rate', 'h_host_7_goal_rate', 
    
    'v_perf_gs', 'v_perf_gd',
    'v_perf_avg_gs', 'v_perf_avg_gd', 'v_host_gs', 'v_host_gd',
    'v_host_avg_gs', 'v_host_avg_gd', 'v_r3_gs', 'v_r3_gd',
    'v_r3_avg_gs', 'v_r3_avg_gd', 'v_perf_bet_high', 'v_perf_bet_low',
    'v_host_bet_high', 'v_host_bet_low', 'v_host_0_1_goal',
    'v_host_2_3_goal', 'v_host_ab_4_goal', 'v_host_0_goal',
    'v_host_1_goal', 'v_host_2_goal', 'v_host_3_goal', 'v_host_4_goal',
    'v_host_5_goal', 'v_host_6_goal', 'v_host_7_goal',
    
    'v_perf_gs_rate', 'v_perf_gd_rate',
    'v_perf_avg_gs_rate', 'v_perf_avg_gd_rate', 'v_host_gs_rate', 'v_host_gd_rate',
    'v_host_avg_gs_rate', 'v_host_avg_gd_rate', 'v_r3_gs_rate', 'v_r3_gd_rate',
    'v_r3_avg_gs_rate', 'v_r3_avg_gd_rate', 'v_perf_bet_high_rate', 'v_perf_bet_low_rate',
    'v_host_bet_high_rate', 'v_host_bet_low_rate', 'v_host_0_1_goal_rate',
    'v_host_2_3_goal_rate', 'v_host_ab_4_goal_rate', 'v_host_0_goal_rate',
    'v_host_1_goal_rate', 'v_host_2_goal_rate', 'v_host_3_goal_rate', 'v_host_4_goal_rate',
    'v_host_5_goal_rate', 'v_host_6_goal_rate', 'v_host_7_goal_rate',

    'h_perf_win_count', 'h_perf_draw_count', 'h_perf_lose_count',
    'h_host_win_count', 'h_host_draw_count', 'h_host_lose_count',
    'h_battle_with_front_10_win_count',
    'h_battle_with_front_10_draw_count',
    'h_battle_with_front_10_lose_count',
    'h_battle_with_end_10_win_count',
    'h_battle_with_end_10_draw_count',
    'h_battle_with_end_10_lose_count', 'v_perf_win_count',
    'v_perf_draw_count', 'v_perf_lose_count', 'v_host_win_count',
    'v_host_draw_count', 'v_host_lose_count',
    'v_battle_with_front_10_win_count',
    'v_battle_with_front_10_draw_count',
    'v_battle_with_front_10_lose_count',
    'v_battle_with_end_10_win_count',
    'v_battle_with_end_10_draw_count',
    'v_battle_with_end_10_lose_count',
    
    'h_perf_win_count_rate', 'h_perf_draw_count_rate', 'h_perf_lose_count_rate',
    'h_host_win_count_rate', 'h_host_draw_count_rate', 'h_host_lose_count_rate',
    'h_battle_with_front_10_win_count_rate',
    'h_battle_with_front_10_draw_count_rate',
    'h_battle_with_front_10_lose_count_rate',
    'h_battle_with_end_10_win_count_rate',
    'h_battle_with_end_10_draw_count_rate',
    'h_battle_with_end_10_lose_count_rate', 'v_perf_win_count_rate',
    'v_perf_draw_count_rate', 'v_perf_lose_count_rate', 'v_host_win_count_rate',
    'v_host_draw_count_rate', 'v_host_lose_count_rate',
    'v_battle_with_front_10_win_count_rate',
    'v_battle_with_front_10_draw_count_rate',
    'v_battle_with_front_10_lose_count_rate',
    'v_battle_with_end_10_win_count_rate',
    'v_battle_with_end_10_draw_count_rate',
    'v_battle_with_end_10_lose_count_rate',
    
    'h_count', 'h_all_count',
    'v_count', 'v_all_count',
    
    'h_current_win_count',
    'h_current_draw_count', 'h_current_lose_count', 
    'h_current_win_count_rate',
    'h_current_draw_count_rate', 'h_current_lose_count_rate', 
    
    'h_current_score',
    'v_current_win_count', 'v_current_draw_count',
    'v_current_lose_count', 
    
    'v_current_win_count_rate', 'v_current_draw_count_rate',
    'v_current_lose_count_rate',
    
    'v_current_score',
    'h_current_rank', 'v_current_rank']
    
params={
    'booster':'gbtree',
    # 这里手写数字是0-9，是一个多类的问题，因此采用了multisoft多分类器，
    'objective': 'multi:softmax', 
    'num_class':3, # 类数，与 multisoftmax 并用
    'gamma':0.1,  # 在树的叶子节点下一个分区的最小损失，越大算法模型越保守 。[0:]
    'max_depth':8, # 构建树的深度 [1:]
    #'lambda':450,  # L2 正则项权重
    'subsample':0.7, # 采样训练数据，设置为0.5，随机选择一般的数据实例 (0:1]
    'colsample_bytree':0.7, # 构建树树时的采样比率 (0:1]
    #'min_child_weight':12, # 节点的最少特征数
    'silent':1 ,
    
#     这部分需要调整
#     'eta': 0.05, # 如同学习率
    'eta': 0.01, # 如同学习率
    
    
    'seed':710,
    'nthread':4,# cpu 线程数,根据自己U的个数适当调整
}

train_dataset = train_dataset_df[train_dataset_df['year'] < 2019]
test_dataset = train_dataset_df[train_dataset_df['year'] == 2019]

valid_dataset = test_dataset[test_dataset['month'] < 3]
test_dataset = test_dataset[test_dataset['month'] >= 3]

xgtrain = xgb.DMatrix(train_dataset[x_columns], label=train_dataset['fix_result'])
xgtest = xgb.DMatrix(test_dataset[x_columns], label=test_dataset['fix_result'])
xgvalid = xgb.DMatrix(valid_dataset[x_columns], label=valid_dataset['fix_result'])

watchlist = [(xgtrain, 'train'),(xgvalid, 'val')]

# num_rounds = 10000
# stop_rounds = 100

num_rounds = 10000
stop_rounds = 300


model = xgb.train(params, xgtrain, num_rounds, watchlist,early_stopping_rounds=stop_rounds)
print(model.best_iteration)

preds = model.predict(xgtest, ntree_limit=model.best_iteration)
preds

[1562]	train-merror:0.039688	val-merror:0.236407
[1563]	train-merror:0.039688	val-merror:0.236407
[1564]	train-merror:0.039584	val-merror:0.236407
[1565]	train-merror:0.039479	val-merror:0.235225
[1566]	train-merror:0.039479	val-merror:0.235225
[1567]	train-merror:0.039427	val-merror:0.235225
[1568]	train-merror:0.039401	val-merror:0.237589
[1569]	train-merror:0.039061	val-merror:0.236407
[1570]	train-merror:0.038982	val-merror:0.235225
[1571]	train-merror:0.039009	val-merror:0.237589
[1572]	train-merror:0.039009	val-merror:0.237589
[1573]	train-merror:0.03893	val-merror:0.236407
[1574]	train-merror:0.038904	val-merror:0.236407
[1575]	train-merror:0.038878	val-merror:0.236407
[1576]	train-merror:0.038747	val-merror:0.236407
[1577]	train-merror:0.038616	val-merror:0.235225
[1578]	train-merror:0.038643	val-merror:0.235225
[1579]	train-merror:0.038512	val-merror:0.236407
[1580]	train-merror:0.038512	val-merror:0.236407
[1581]	train-merror:0.038381	val-merror:0.237589
[1582]	train-merror:0

array([2., 0., 0., ..., 2., 1., 1.], dtype=float32)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(test_dataset['fix_result'], preds)

0.7986627043090639

In [27]:
submission = pd.DataFrame({
    "matchid": test_dataset.matchid, 
    "result": test_dataset.fix_result,
    "pred": preds
})

submission.to_csv('../data/submission_2019.csv', index=False)

In [28]:
model.save_model('../model/xgb.model')

# 预测

In [39]:
# 确认预测轮次
issue = 19062

## 获取预测比赛列表

In [40]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_lottery_predict_info` where issue = %d;' % (issue)
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    predict_game_df = pd.DataFrame(rows)
    
    connection.close()

In [41]:
predict_game_df = predict_game_df[(predict_game_df['game'].isin(match_group))]
predict_game_df = predict_game_df.dropna()

In [42]:
predict_game_df['year'] = predict_game_df.apply(lambda row: row.time.year, axis=1)
predict_game_df['month'] = predict_game_df.apply(lambda row: row.time.month, axis=1)
predict_game_df['day'] = predict_game_df.apply(lambda row: row.time.day, axis=1)

In [43]:
predict_game_df = encode_team(predict_game_df)
predict_game_df = encode_game(predict_game_df)

## 获取预测特征数据

In [44]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ml', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_football_feature_info` where matchid in (select matchid from `breadt_lottery_predict_info` where issue = %d);' % (issue)
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    predict_feature_df = pd.DataFrame(rows)
    
    connection.close()

In [45]:
predict_feature_df = take_count(predict_feature_df)
predict_feature_df = take_rate(predict_feature_df)
predict_feature_df = take_score(predict_feature_df)

predict_feature_df['h_current_rank'] = predict_feature_df.apply(lambda row: get_current_rank(row.h_rank), axis=1)
predict_feature_df['v_current_rank'] = predict_feature_df.apply(lambda row: get_current_rank(row.v_rank), axis=1)

predict_feature_df = take_goal_info('h', predict_feature_df)
predict_feature_df = take_goal_info('v', predict_feature_df)

## 合并预测数据

In [46]:
predict_dataset_df = pd.merge(predict_game_df, predict_feature_df, on='matchid', how='left')
predict_dataset_df = predict_dataset_df.dropna()
predict_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Columns: 223 entries, draw_bet_return to v_host_7_goal_rate
dtypes: datetime64[ns](1), float64(99), int64(90), object(33)
memory usage: 24.5+ KB


In [47]:
xgpredict = xgb.DMatrix(predict_dataset_df[x_columns])

official_preds = model.predict(xgpredict, ntree_limit=model.best_iteration)
official_preds

array([1., 2., 0., 0., 2., 2., 2., 1., 2., 1., 0., 2., 0., 0.],
      dtype=float32)

In [48]:
pd.DataFrame({
    "matchid": predict_dataset_df.matchid, 
    "home_team": predict_dataset_df.home_team, 
    "visit_team": predict_dataset_df.visit_team, 
    "game": predict_dataset_df.game, 
    "win_bet_return":predict_dataset_df.win_bet_return,
    "draw_bet_return":predict_dataset_df.draw_bet_return,
    "lose_bet_return":predict_dataset_df.lose_bet_return,
    "pred": official_preds
})

,matchid,home_team,visit_team,game,win_bet_return,draw_bet_return,lose_bet_return,pred
0,2406953,阿雅克肖,奥尔良,法乙,1.75,3.55,4.48,1.0
1,2406952,勒阿弗尔,阿雅GFCO,法乙,1.79,3.35,4.55,2.0
2,2406947,欧塞尔,沙托鲁,法乙,1.97,3.11,4.01,0.0
3,2406950,贝济耶,布雷斯特,法乙,4.17,3.44,1.82,0.0
4,2406946,特鲁瓦,南锡,法乙,2.13,3.15,3.44,2.0
5,2406951,梅斯,瓦朗谢纳,法乙,1.51,4.09,5.89,2.0
6,2406949,索肖,圣旺红星,法乙,1.57,3.62,6.15,2.0
7,2406954,巴黎FC,格勒诺布,法乙,1.81,3.12,4.86,1.0
8,2406538,登博思,前进之鹰,荷乙,2.00,3.72,3.14,2.0
9,2406533,多德勒支,乌德青年,荷乙,1.57,4.43,4.44,1.0
